# Intro

## Statistical and Dynamical Downscaling
Global Climate Models, also referred to as GCMs, are incredibly valuable tools in that they can capture large scale climate dynamics (e.g. future greenhouse gas concentration over the entire Earth).  Furthermore, many of these phenomena must be studied at a global scale.  However, there are limitations to GCMs.  Primarily, given the large scale analysis, grid cells and computation points for these models are actually quite large.  This leads to problems when trying to study local phenomena, such as the climate of a specific city or region.  This is when downscaling must be used.  There are two types of downscaling: Dynamical and Statistical. 

### Dynamical Downscaling
This type of downscaling uses a Regional Climate Model (RCM) which takes in the output of a GCM as a boundary condition.  Then, RCM use these boundary conditions and physical principles to generate higher resolution local climate.  However, these are computationally expensive.

### Statistical Downscaling
Statistical Downscaling is another common method dependent on the availability of local weather data, such as precipitation.  A statistical relationship is developed between the historic observed climate data and the output of the climate model for the same historical period.  Then, this statistical relationship is used to downscale the climate model output to the local scale.  This can be combined with bias correction and adjustment to improve the accuracy of the downscaled data.

For this project, I will be using statistical downscaling.  I will use a Julia package, RainFARM, to downscale coarse precipitation data to a finer scale.

### Statistical Downscaling Methods
Statistical Downscaling, SD, consists of a identified historical period (calibration) used to analyze the predictor/predictand relationship.  This relationship is then applied to predictors from the GCMs to get the local climate projections. 

:::{.callout-note}
Sources include the Copernicus Climate Change Service (C3S), National Oceanic and Atmospheric Administration (NOAA), Evaluaation of statistical downscaling methods for climate change projections over Spain: Present conditions with perfect predictors (2021).
:::

# Project
For this project, I will be conducting a slightly different analysis than others in the class.  As I proposed to Dr. Doss-Gollin, I want to undertake a deep dive into a Julia based Downscaling Package, RainFARM, analyze the methods and algorithms used, and apply it to the larger scale precipitation data.

# RainFARM
According to Rebora et al. (2006b), RainFARM aims to generate "an ensemble of stochastic realizations of the small-scale precipitation fields that have statistical properties similar to those measured for rainfall in a given area and/or synoptic situation."  In other words, RainFARM takes course data and downscales it to a finer scale while still keeping similar statistical properties (e.g. it presents a faithful representataion of what your data could look like on the finer scale).  However, it is important to note that it is a stochastic process, and randomness is involved.  Consequently, it is not a deterministic process, and the results will vary each time the code is run.  This is why it is important to run the code multiple times to see many different possible downscaled outcomes.

## Common Rainfall Downscaling Methods Overview
There are many different methods used for downscaling, but there are three main types used in the literature, according to the Rebora article.  These are:
1. Point Processes
2. Autoregressive Processes with Nonlinear Transformation: This involves using patterns from the past (autoregressive) and applying it to a static nonlinear transformation.
3. Fractal Cascades: This creates a model where rain repeats in patterns inspired by fractals.
However, there have been examples of people combining these methods to create hybrid models.

## Methodology Outline
The RainFARM methodology uses a two step process:
1. Linear Autoregressive Process: The behavior of the rainfall on the course scale can be analyzed for patterns over this large area.  A simple forecast is made from this step.
2. Nonlinear Filtering: This is applied to the baseline prediction, and it adds detail and makes adjustments to increase accuracy.
This methodology works well with overall weather patterns.

## Inputs, Outputs, and Variables

### Inputs
RainFARM can input a course rainfall field P(X, Y, T).  This type of data can come from a few different sources, like GCMs, but it lacks the resolution at a fine scale needed for local analysis and regional studies.  X and Y represent the latitude and longitude of the grid on which data is stored, and T indicates time.  

$$
P(X, Y, T) = Input
$$
$$
L_{0} <= (X,Y) <= L_{max}
$$
$$
T_{0} <= T <= T_{max}
$$

The second two equations above define the range of scales for L and T.  These ranges refer to the scale of the data.  L0 and T0 are defined at the scales on which the data forecast becomes poor.  Typically, the spatial forecase becomes poor around 10-20 km, and the temporal forecast becomes poor around 2-4 hours (for the types of data on which RainFARM is used).

### Outputs

The goal of the RainFARM package is to output a new precipitation field, written as r(x,y,t).  The new scale, lambda, will be less than the original scale, L, and typically falls around  1-2 km.  The new time scale, tau, will be less than the original time scale, T, and typically falls around 10 minutes.

This output must be thought of as one potential realization of the small scale behavior of this larger system and phenomena.

$$
r(x, y, t) = output 
$$
$$
\lambda_{0} < L_{0}
$$
$$
\tau_{0} < T_{0}
$$

### Aggregation and Validation

Interestingly, RainFARM includes its own self validation in which the output, the fine scale precipitation field, r(x,y,t), is aggregated on the original spatial and temporal scales.  This aggregated field should produce a coarse scale precipitation field, R(X, Y, T) that must be equal to the original field P.

## Detailed Methodology

### Fourier Analysis & Power Spectrum
The Rebora paper first describes the estimation of the space-time power spectrum of the field P.  This space-time power spectrum can be thought of as the power distribution of the field P.  It helps describe how precipitation is generated across space and time, and is frequency used in meteorology.

To better understand this, we can go into more detail on both Fourier analysis and Power Spectra.

#### Fourier Analysis
Fourier Analysis was first developed by French mathematician Joseph Fourier.  It is a mathematical tool that can break down complex signals into a sum of simpler sinusoidal (wave-like) functions.  Consider, for example, a recording of a song.  Fourier Analysis could isolate the individual notes that make up the song by analyzing the frequency components.  

Fourier analysis itself is complex, but it can be considered a method to break down complex components into simpler ones.  Therefore, we can analyze the complex patterns within the precipitation field and break them down into frequency components we can analyze easily.

It is also important to note that these frequency components have an associated power which tells us how impactful they are in the system.  This is where the power spectrum comes in.

#### Power Spectra
The power spectrum Sxx(f) of a time series x(t) describes the distribution of power into frequency components composing that signal.  The statistical average of any sort of signal as analyzed in terms of its frequency content is its spectrum.  This is frequently done as analysis over a frequency-domain representation is easier to analyze than a time-domain representation.

Below, I go through the steps of the RainFARM methodology set up in the Rebora paper.  First, the space-time power spectrum is defined, and Ky and Kx represent wave numbers and Omega represents angular frequency.  The Nyquist Number is also defined, and it is the maximum frequency that can be represented given the sampling frequency.  This is important because it defines the limitations of the analysis.

$$
|\hat{P}(K_{X}, K_{Y},\Omega|^2 = space \space time \space power \space spectrum
$$
$$
K_{X} \space and \space K_{Y} = wavenumbers \space in \space X \space and \space Y \space directions
$$
$$
\Omega = angular \space frequency
$$
$$
(K_{X}, K_{Y}) <= pi/L_{0} = Nyquist \space Number
$$
$$
\Omega <= pi/T_{0} = Nyquest \space Frequency
$$

So we have an equation for the space time power spectrum above.  In addition, we have representations of our wavenumbers where the wavenumbers are the components of a wavevector along the x and y directions in the case of a two-dimensional setting.  Wavenumbers can be thought of as the spatial frequency of a wave.  In other words, the wavenumbers allow for a representation of the spatial frequency of these energy waves.  K(X) and K(Y) represent how these precipitation patterns vary in the X and Y directions.  However, the Nyquist Number defines the limitations of our understanding, but an in-depth discussion of this determination is outside the scope of this work.

The angular frequency, in the general theory, measures how fast the waves oscillate in time.  In other words, it represents how fast the pattern changes, so for RainFARM it represents how quickly the precipitation patterns change.  Again, the Nyquist Frequency defines the limitations of the analysis.

### Downscale Power Spectrum
Next, the power spectrum is extrapolated to small scales.  In this way, the downscaling occurs in the frequency domain not the spatial domain.  This is done through the assumption of a power-law behavior.

#### Power-Law Behavior
According to a helpful Wikipedia, a power-law behavior is a "functional relationship between two quantities where a relative change in one quantity results in a relative change in the other quantity proportional to a power of the change, independent of the initial size of those quantities: one quantity varies as a power of another."  This may appear to be a large assumption, but certain studies (e.g. Ferraris et al. 2003a, b) have shown that a power-law behavior is a reasonable assumption for precipitation fields.

#### Slopes
Once we have a downscaled precipitation field (using the assumption that there is aa functional relationship between the coarse precipitation field and the fine field in the frequency domain), the spatial and temporal logarithmic slopes of the power spectrum, which we call α and β, are defined. These slopes express the rate at which the power of the frequency components changes with respect to space and time.  For example, a high spatial logarithmic slope means that the power dynamics can chanage quickly as you move across the field.  In addition, isotropy is assumed in the two spatial directions.  This is also a valid assumption with precipitation. This uniform behavior in all directions lessens the computational load and simplifies the model.

### Fourier Spectrum Inversion
We have been conducting analysis in the frequency domain, but another useful product of Fourier Analysis is that this analysis can be inverted to change back into the space time domain.  This can be broken up into the definition of the Fourier Spectrum, the Power Spectrum, and the Inversion.

A Fourier spectrum is defined below. This Fourier analysis lends itself to an exploration of the frequency components of the data and a consequent modeling of the distribution of power across frequencies.

$$
\hat{g}(k_{x},k_{y},\omega) = |\hat{g}(k_{x}, k{y}, \omega)|e^{i\phi}
$$

A Power spectrum is defined below.  This functional equation allows us to describe the power spectrum in a mathematical form.  

$$
|\hat{g}(k_{x}, k{y}, \omega)|^2 \propto (k_{x}^2 + k_{y}^2)^{-\alpha/2} \omega^{-\beta}
$$

The next step is inversion. The power spectrum can be inverted to generate a Gaussian field that represents the data in space time.  A Gaussian field is a random field involving Gaussian probability density functions of the variables.  Gaussian fields are often used to model random processes that vary smoothly in space or time. 

### Generate a Precipitation Field
Finally, a nonlinear transformation, defined below, is applied to all points in the Gaussian field g.  This produces a synthetic precipitation field r(x, y, t). 

$$
\tilde{r}=exp[g(x,y,t)]
$$

Unfortunately, no documentation online gives the rational for selecting this method of nonlinear transformation.  However, it is a relatively simple nonlinear transformation which minimizes computational load.

### Summary and Validation
At this point in the process, (1) a coarse data field has been taken in, (2) a power spectrum has been generated to represent the energy/precipitation in the atmosphere, (3)  the power spectrum is extrapolated to smaller scales with the assumption of power-law behavior, (4) fourier spectrum are used to create a function that represents this downscaled power spectrum and then inverted to create a gaussian field that brings the downscaled data back to a space time field. Finally, a precipitation field is generated by taking "a nonlinear transformation of the Gaussian field g" (Rebora et al., 2006b). 

As mentioned earlier, there is an internal validation step in which the generated (or synthetic) precipitation field is compared to the original field.  This is done by aggregating the synthetic field to the original scale; in other words, the small scale field is aggregated to L0 and T0.

In each box, the ratio of the original field, P, and the synthetic field, R, is calculated.  If the ratio is 1, there is a perfect match.  If it is not one, the synthetic field R can be adjusted.  This means that the synthetic field closely resembles the original field at the L0 and T0 scale.

## Important Notes
As discussed, the RainFARM methodology is stochastic, and this random behavior is associated with the Fourier phases. The phases are randomly generated, and this randomness is what creates the different realizations of the small scale precipitation field.  This is why it is important to run the code multiple times to see many different possible downscaled outcomes.  The original paper introducing this method writes: "The RainFARM approach is easily used for ensemble predictions; by choosing different sets of random Fourier phases, one can generate a large number of stochastic fields that are all equal to P when aggregated on space and time scales larger than L0 and To, and are different, but with similar statistical properties, on smaller scales."

# Packages


In [ ]:
using Plots
using StatsBase
using StatsPlots
using Distributions
using Statistics
using CDSAPI
using NCDatasets
using RainFARM
using DataFrames
using GLM
using FFTW

# Downloading Data


In [ ]:
precip_tx = NCDataset("data/precip_tx.nc")
precip_tx_p = precip_tx["precip"]
filled_precip_tx_p = coalesce.(precip_tx_p,0)

# Running the RainFARM Function
Now inputs to the RainFARM function must be determined, and they are informed by information from the methodology.

The first input into the function is the coarse precipitation data, or the coarse data P(X, Y, T).  In this case, the coarse data is the precipitation over Texas.

Te second input is the slope.  The slope is the spatial spectral slope.  This value helps quantify the rate at which the power in the precipitation field decreases as you move to small scales in space.

The final input is nf, which is the ratio of coarse to fine data.  For example, if nf=5 the downscaled data will have 5 times the resolution of the original data.

Below, the RainFARM function is called on the precipitation data. Code labels have been added below:

1. RainFARM needs boolean context inputs, and any missing values will come up as "Missing," which is non-boolean.  Consequently, the "Missings" need to be removed.  To do so, the missing values were assumped to be no precipitation, so the default rain is set to 0.
2. The default rain value of no precipitation is used to fill in the missing values.  This is the field fed into the function.
3. The spatial slope is set for 1.  This slope is explored in more detail later.
4. The nf determines the ratio of coarse to fine data.  For example, if nf=5 the downscaled data will have 5 times the resolution of the original data.  It is also explored later.


In [ ]:
using RainFARM
using DataFrames
slope = 1; # <3>
nf = 5; # <4>
r = filled_precip_tx_p; # <2>
global downscaled_precip_tx = rainfarm(r, slope, nf)

This created a variable, downscaled_precip_tx, with the downscaled precipitation data over the same time period of the original precipitation data.

# Fourier Analysis in Code
I have included some examples of how the Fourier Analysis is used to help better understand the methodology used in the rainfarm package.  

The first example is an example created off of a randomly generated sin wave with multiple frequency components.


In [ ]:
using FFTW
using Plots

t0 = 0              # Start time 
fs = 44100          # Sampling rate (Hz)
tmax = 0.1          # End time       

randfreq1 = rand(1:90)
randfreq2 = rand(1:90)

t = t0:1/fs:tmax;   
signal = sin.(2π * randfreq1 .* t) + sin.(2π * randfreq2 .* t) # Signal

F = fftshift(fft(signal))
freqs = fftshift(fftfreq(length(t), fs))

# plots 
time_domain = plot(t, signal, title = "Signal", label="f = sin(2pi $randfreq1 t) + sin(2pi $randfreq2 t)",legend=:top)
freq_domain = plot(freqs, abs.(F), title = "Spectrum", xlim=(0, +100), xticks=-0:20:100, label="Frequency Components",legend=:top) 
freq_domain = vline!([randfreq1], label=false)
freq_domain = vline!([randfreq2], label=false, color=:red)
plot(time_domain, freq_domain, layout = (2,1))

This next example selects a random row and time step in our precipitation field to visualize how the Fourier transform is applied to the data we have.


In [ ]:
randomtime = rand(1:8760)
default_value = 0.0 # <1>
oneslice = filled_precip_tx_p[:,:,randomtime]
plot()
for i = 1:13
    t0 = 0
    fs = 24-1
    tmax = 1
    t = t0:1/fs:tmax;
    signal = oneslice[i,:]
    F = fftshift(fft(signal))
    freqs = fftshift(fftfreq(length(t),fs))
    time_domain = plot(t, signal, title = "One Row of Downscaled Precipitation Data at t = 1", xlabel="Time")
    freq_domain = plot(freqs, abs.(F), title = "Fourier Transform of One Row of Precipitation Data at t=1", xlabel="Frequency")
    global fourierplot = plot!(time_domain, freq_domain, layout = (2,1), legend=false)
end
display(fourierplot)

# Results Analysis

## Coarse Input vs. Fine Output
First, it is possible to analyze the original, coarse data and the downscaled data side by side.  This can be done by plotting the data at a specific time frame.  Consider 10 randomly selected precipitation fields.

1. Generate 10 random time frames from our downscaled data.
2. Plot the downscaled data
3. Plot the original data


In [ ]:
tenrandom = rand(1:8760, 10) # <1>
for i in tenrandom
    timeframe = i
    downheatmap = heatmap(downscaled_precip_tx[:, :, timeframe]) # <2>
    origheatmap = heatmap(precip_tx_p[:, :, timeframe]) # <3>
    cumplot = plot(downheatmap, origheatmap, layout = (1, 2), legend = true)
    display(cumplot)
end

Visually, these side-by-side comparisons show reasonable results for the downscaling.  The highest precipitation hot spots are located in the same places across the coarse and fine data, and general spatial patterns are similar.

You can also see this three-dimensionally.  Below, the plots show the downscaled data plotted on the grid. 


In [ ]:
for i = 1:3
    randtime = rand(1:8760)
    xs = 1:120
    ys = 1:120
    zs = downscaled_precip_tx[:,:,randtime]
    sur1 = surface(xs, ys, zs)

    xs = 1:24
    ys = 1:24
    zs = filled_precip_tx_p[:,:,randtime]
    sur2 = surface(xs, ys, zs)

    display(plot!(sur1, sur2, layout = (1, 2), legend = true))
end

## Stochastic Visual Analysis

However, it is also important to remember that this is a stochastic process.  Let's explore the different options that can be generated by this same function.

The code below will generate a random time step and then run the downscaling function two times at this time step.  The two outputs will be plotted side by side to demonstrate that there are variations within these results.  This visually demonstrates that this process is stochastic.

1. Generate a random time step
2. Call the original data on the time step
3. Fix the missing data using a default value
4. Call the downscaling functions multiple times on the same data
5. Plot the results side by side


In [ ]:
using RainFARM
using DataFrames
randomtimestep = rand(1:8760) # <1>
stoch_precip_tx_p = filled_precip_tx_p[:, :, randomtimestep] # <2>
slope = 1;
nf = 5;
downscaled_stoch1_precip_tx = rainfarm(stoch_precip_tx_p, slope, nf) # <4>
downscaled_stoch2_precip_tx = rainfarm(stoch_precip_tx_p, slope, nf) # <4>
stoch1_hm = heatmap(downscaled_stoch1_precip_tx[:, :, 1])
stoch2_hm = heatmap(downscaled_stoch2_precip_tx[:, :, 1])
cumplot = plot(stoch1_hm, stoch2_hm, layout = (1, 2), legend = true) # <5>

We can explore this further below.  The code below plots the downscaled data of a random row at a given time step 100 times to show the variation within these outputs.

1. Generate a random time step
2. Take a random "slice" or row
3. Plot these realizations


In [ ]:
using RainFARM
using DataFrames
plot()
randomtimestep = rand(1:8760) # <1>
r = filled_precip_tx_p[:, :, randomtimestep] 
slope = 1;
nf = 5;
emptymatrixslice = zeros(100, 120)
for i = 1:100
    randslice = rand(1:120) # <2>
    sldownscaled = rainfarm(r, slope, nf)
    downscaledslice = sldownscaled[randslice, :, :]
    global sliceplot = plot!(downscaledslice; legend=false) #<3>
    global emptymatrixslice[i, :] = downscaledslice
end

display(sliceplot)

This next code shows the quantiles of these realizations.  This is a useful way to visualize the variation in the data.  The quantiles are plotted for each time step.


In [ ]:
emptyquantiletrack = zeros(3,120)
for i = 1:120
    global emptyquantiletrack[:, i] = quantile(emptymatrixslice[:, i], [0.25, 0.5, 0.75])
end

quantileplot = plot(emptyquantiletrack[1, :], label = "25th Percentile")
quantileplot = plot!(emptyquantiletrack[2, :], label = "50th Percentile")
quantileplot = plot!(emptyquantiletrack[3, :], label = "75th Percentile")

display(quantileplot)

The code below again shows the stochastic nature of these realizations with a surface plot. 


In [ ]:
randtime = rand(1:8760)
stoch_precip_tx_p = filled_precip_tx_p[:, :, randomtimestep]
slope = 1;
nf = 5;
downscaled_stoch1_precip_tx = rainfarm(stoch_precip_tx_p, slope, nf) 
downscaled_stoch2_precip_tx = rainfarm(stoch_precip_tx_p, slope, nf) 

xs = 1:120
ys = 1:120
zs = downscaled_stoch1_precip_tx[:,:,1]
sur1 = surface(xs, ys, zs)

xs = 1:120
ys = 1:120
zs = downscaled_stoch2_precip_tx[:,:,1]
sur2 = surface(xs, ys, zs)

display(plot!(sur1, sur2, layout = (1, 2), legend = true))

## Spatial Slope Analysis

However, another interesting thing to analyze is the determination of the spatial slope factor.  This is an important component of our frequency analysis of the energy/precipitation in the system and how large this rate of change is.  Let's compare the graphs with different spatial slopes.  Higher spatial slopes correspond to a faster rate of change.

1. Test over a range of slopes between 0.5 and 7, with a step of 0.5.
2. Call the downscaling on each slope value
3. Plot the results


In [ ]:
using RainFARM
using DataFrames
default_value = 0.0
randomtimestep = rand(1:8760)
randtime_precip_tx_p = precip_tx_p[:, :, randomtimestep]
r = coalesce.(randtime_precip_tx_p, default_value)
for i = 0.5:0.5:7 # <1>
    slope = i;
    nf = 5;
    downscaled_randslope_precip_tx = rainfarm(r, slope, nf) # <2>
    randtime_hm = heatmap(downscaled_randslope_precip_tx[:, :, 1]) # <3>
    display(randtime_hm)
end

slopedownscale1 = rainfarm(r,0.1,nf)
slopedownscale2 = rainfarm(r,20,nf)

xs = 1:120
ys = 1:120
zs = slopedownscale1[:,:,1]
sur1 = surface(xs, ys, zs)

xs = 1:120
ys = 1:120
zs = slopedownscale2[:,:,1]
sur2 = surface(xs, ys, zs)

display(plot!(sur1, sur2, layout = (1, 2), legend = true))

As the slope grows, there is a 'blurring' effect on the data.  The smallest slope generates the most specific estimate and is the least blurry of our predictions.  This generates an interesting question.  The available RainFARM documentation does not give many recommendations regarding this spatial slope, and the issue, therefore, becomes a question of variance.  With a small slope, there is a small rate of change the predictions are very specific and localized.  With a high slope, there is a high rate of change so the predictions become blurry and less specific.  Therefore, low slopes have low variance and high slopes have high variance.  Low variance is great when the model is accurate, but it can cause problems with generalization.  High variance can generalize well, but the accuracy is limited.  This is an important question to consnider when setting up this RainFARM model.

## Coarse to Fine Ratio Analysis

We can also consider the ratio of coarse to fine data, or the nf variable.  This variable determines the resolution of the downscaled data.  Let's compare the graphs with different nf values.  Higher nf values correspond to a higher resolution.

1. Call on your random time step
2. Test a range of nf values between 1 and 30, with a step of 2.


In [ ]:
using RainFARM
using DataFrames
default_value = 0.0
randomtimestep = rand(1:8760)
randtime_precip_tx_p = precip_tx_p[:, :, randomtimestep] # <1>
r = coalesce.(randtime_precip_tx_p, default_value)
for i = 1:2:30 # <2>
    slope = 1;
    nf = i;
    downscaled_randnf_precip_tx = rainfarm(r, slope, nf)
    randnf_hm = heatmap(downscaled_randnf_precip_tx[:, :, 1])
    display(randnf_hm)
end

This analysis starts with nf = 1, meaning the output is the original coarse data.  As nf grows, the downscaled data becomes increasingly fine.  However, there does seem to be a potential point at which the downscaled data is not improving and increasing the ratio only serves to diminish the hotspot peaks of the precipitation.  In the heatmap, this can be seen as the downscaled patterns stop changing as much as the ratio increases, but the hotspots dim.  This is another important question to consider when setting up this RainFARM model.

# Conclusions

## Considerations

Spatial Slope: The choice of spatial slope introduces a trade-off between specificity and generalization. Low slopes provide specific but localized estimates, while high slopes result in more generalized but blurry predictions.

Coarse to Fine Ratio (nf): The analysis indicates a potential point of diminishing returns for increasing the ratio, where fine-scale details may be lost.
Recommendations:

## Model Performance

The RainFARM Model seemed to make good predictions and acknowledges the sources of error in precipitation data.  These stocastic predictions represent the statistical trends of the coarse data well.  I believe it is a useful model, and I hope to use it for my thesis later!